In [36]:
import numpy as np

def viterbi_log(A, C, B, O):
    """Viterbi algorithm (log variant) for solving the uncovering problem

    Args:
        A (np.ndarray): State transition probability matrix of dimension I x I
        C (np.ndarray): Initial state distribution  of dimension I
        B (np.ndarray): Output probability matrix of dimension I x K
        O (np.ndarray): Observation sequence of length N

    Returns:
        S_opt (np.ndarray): Optimal state sequence of length N
        D_log (np.ndarray): Accumulated log probability matrix
        E (np.ndarray): Backtracking matrix
    """
    I = A.shape[0]    # Number of states
    N = len(O)  # Length of observation sequence

    # Initialize D and E matrices
    D = np.zeros((I, N))
    E = np.zeros((I, N-1)).astype(np.int32)
    D[:, 0] = np.multiply(C, B[:, O[0]])

    # Compute D and E in a nested loop
    for n in range(1, N):
        for i in range(I):
            temp_product = np.multiply(A[:, i], D[:, n-1])
            D[i, n] = np.max(temp_product) * B[i, O[n]]
            E[i, n-1] = np.argmax(temp_product)

    # Backtracking
    S_opt = np.zeros(N).astype(np.int32)
    S_opt[-1] = np.argmax(D[:, -1])
    for n in range(N-2, -1, -1):
        S_opt[n] = E[int(S_opt[n+1]), n]

    return S_opt, D, E


A = np.array([[0.8, 0.1, 0.1], 
              [0.2, 0.7, 0.1], 
              [0.1, 0.3, 0.6]])

C = np.array([0.6, 0.2, 0.2])

B = np.array([[0.7, 0.0, 0.3], 
              [0.1, 0.9, 0.0], 
              [0.0, 0.2, 0.8]])


O = np.array([0, 2, 0, 2, 2, 1]).astype(np.int32)
#O = np.array([1]).astype(np.int32)
#O = np.array([1, 2, 0, 2, 2, 1]).astype(np.int32)

# Apply Viterbi algorithm (log variant)
S_opt, D_log, E = viterbi_log(A, C, B, O)

print('Observation sequence:   O = ', O)
print('Optimal state sequence: S = ', S_opt)
np.set_printoptions(formatter={'float': "{: 7.2f}".format})
print('D_log =', D_log, sep='\n')
np.set_printoptions(formatter={'float': "{: 7.4f}".format})
print('exp(D_log) =', np.exp(D_log), sep='\n')
np.set_printoptions(formatter={'float': "{: 7.0f}".format})
print('E =', E, sep='\n')

Observation sequence:   O =  [0 2 0 2 2 1]
Optimal state sequence: S =  [0 0 0 2 2 1]
D_log =
[[   0.42    0.10    0.06    0.01    0.00    0.00]
 [   0.02    0.00    0.00    0.00    0.00    0.00]
 [   0.00    0.03    0.00    0.00    0.00    0.00]]
exp(D_log) =
[[ 1.5220  1.1061  1.0581  1.0136  1.0033  1.0000]
 [ 1.0202  1.0000  1.0010  1.0000  1.0000  1.0006]
 [ 1.0000  1.0342  1.0000  1.0045  1.0022  1.0003]]
E =
[[0 0 0 0 0]
 [0 0 0 0 2]
 [0 2 0 2 2]]


In [78]:
def VITERBI(state_transition_probmat, observation_probmat, observation_sequence, initial_state_prob):
    state_transition_probmat = np.array(state_transition_probmat)
    observation_probmat = np.array(observation_probmat)
    initial_state_prob = np.array(initial_state_prob)
    observation_sequence = observation_sequence.astype(int)

    viterbi_mat = np.empty((state_transition_probmat.shape[1],len(observation_sequence)))
    backpointer = np.empty((state_transition_probmat.shape[1],len(observation_sequence)-1)).astype(int)
    for i in range(state_transition_probmat.shape[1]):
        viterbi_mat[i,0] = float(initial_state_prob[i]*observation_probmat[i,observation_sequence[0]])
    #     print(viterbi_mat[i,0])
    # print(viterbi_mat)
    # viterbi_mat[:,0] = np.multiply(initial_state_prob, observation_probmat[:,observation_sequence[0]])
    for time_step in range(1,len(observation_sequence)):
        for state in range(state_transition_probmat.shape[1]):
            state_vec = viterbi_mat[:,time_step-1]*state_transition_probmat[:,state,time_step-1]
            state_vec = state_vec*observation_probmat[state,observation_sequence[time_step]]
        
            viterbi_mat[state,time_step] = np.max(state_vec)
            backpointer[state,time_step-1] = np.argmax(state_vec)
    best_path_prob = np.max(viterbi_mat[:,-1])
    best_backpointer = np.argmax(viterbi_mat[:,-1])
    best_path = [best_backpointer]
    j = 0
    for i in reversed(range(len(observation_sequence)-1)):
        best_path.append(backpointer[best_path[j],i])
        j += 1
    best_path = best_path[::-1]
    return best_path, viterbi_mat

In [44]:
A = np.array([[0.8, 0.1, 0.1], 
              [0.2, 0.7, 0.1], 
              [0.1, 0.3, 0.6]])

C = np.array([0.6, 0.2, 0.2])

B = np.array([[0.7, 0.0, 0.3], 
              [0.1, 0.9, 0.0], 
              [0.0, 0.2, 0.8]])


O = np.array([0, 2, 0, 2, 2, 1]).astype(np.int32)

best_path, viterbi_path =VITERBI(A,B,O,C)
print(best_path)
np.set_printoptions(formatter={'float': "{: 7.2f}".format})

print("viterbi_path: ", viterbi_path)

[np.int64(0), np.int64(0), np.int64(0), np.int64(2), np.int64(2), np.int64(1)]
viterbi_path:  [[   0.42    0.10    0.06    0.01    0.00    0.00]
 [   0.02    0.00    0.00    0.00    0.00    0.00]
 [   0.00    0.03    0.00    0.00    0.00    0.00]]


In [79]:
A = np.array([[3.0, 1.0, 2.0], 
              [4.0, 2.0, 0.0], 
              [3.0, 2.0, 1.0]])
B = np.array([[0.0,1.0,2.0],[1.0,2.0,1.0],[1.0,3.0,1.0]])


new_arr = np.stack((A,B),axis = 2)
print(new_arr.shape)
C = np.array([1.0,1.0,1.0])

B = np.array([[1.0], 
              [1.0], 
              [1.0]])


O = np.array([0,0,0]).astype(np.int32)

print(A.shape)
print(B.shape)
print(O.shape)
best_path, viterbi_path =VITERBI(new_arr,B,O,C)
print(best_path)
np.set_printoptions(formatter={'float': "{: 7.2f}".format})

print("viterbi_path: ", viterbi_path)

(3, 3, 2)
(3, 3)
(3, 1)
(3,)
[np.int64(1), np.int64(0), np.int64(2)]
viterbi_path:  [[   1.00    4.00    2.00]
 [   1.00    2.00    6.00]
 [   1.00    2.00    8.00]]


In [ ]:
#viterbi and Transformer 

